In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
import os
import fnmatch
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [2]:
#read in the file and create dataframe
#excplicitly state the part
#path = r'C:\Users\einot\OneDrive\Documents\CERN VScode\CERN-Python\4K desorption measurements\Cu sample\EGA fully in\S1 no preinjection at cold temps'
#mid = pd.read_csv(os.path.join(path, "MID.tsv"), sep="\t", skiprows=lambda x: x<=11, on_bad_lines="skip")

#List molecule masses used in MID scan

#masslist = [2,4,12,15,16,18,28,32,40,44]

#Define datetime format
#MID_datetime = "%Y/%m/%d %H:%M:%S.%f"

#Reformat time values
def timeformat(df,format):
    new_df = df.copy()
    for idx, col in enumerate(df):
        if idx % 2 == 0:  # Check if the column index is even (every other column)
            pandas_timestamp = pd.to_datetime(df[col], format=format)
            timestamp = pandas_timestamp.dt.strftime('%d-%m-%Y %H:%M:%S')
            new_df[col] = timestamp  # Replace the time column in the new dataframe
    return new_df

#call the function with correct parameters to modify MID dataframe
#mid = timeformat(mid,MID_datetime)

#### Vaclogger

In [3]:
""" #Reading in the file
vaclog = pd.read_csv(os.path.join(path, "vaclog"), sep="\t")

#Define original datetime format
vaclog_datetime = "%d/%m/%Y %H:%M:%S"
 """
#Adding an elapsed time column for temperature fits
def elapsed_time(df,timecol,format):
    new_df = df.copy()
    pandas_timestamp = pd.to_datetime(new_df[timecol],format=format)
    runtime = (pandas_timestamp-pandas_timestamp[0]).dt.total_seconds()
    insert_idx = df.columns.get_loc(timecol) + 1  # Get the index to insert the new column
    new_df.insert(insert_idx,"Elapsed time", runtime)
    reformat = pandas_timestamp.dt.strftime('%d-%m-%Y %H:%M:%S') #New timestamp format for plotting
    print(reformat)
    new_df[timecol] = reformat
    new_df[timecol] = pd.to_datetime(new_df[timecol],format='%d-%m-%Y %H:%M:%S') #convert to pd datetime
    
    return new_df
""" 
#call the function with correct parameters to modify vaclogger dataframe
vaclog = elapsed_time(vaclog,"Time",vaclog_datetime) """

' \n#call the function with correct parameters to modify vaclogger dataframe\nvaclog = elapsed_time(vaclog,"Time",vaclog_datetime) '

In [4]:
def process_data(root):
    #Define datetime formats
    MID_datetime = "%Y/%m/%d %H:%M:%S.%f"
    vaclog_datetime = "%d/%m/%Y %H:%M:%S"
    hv_datetime = "%d/%m/%Y %H:%M:%S.%f"
    
    #Create an empty dictionary to store processed dataframes
    dataframes = {}

    # Iterate over the subdirectories starting from the specified directory
    for dirpath, dirs, files in os.walk(root):
        for filename in files:
            filepath = os.path.join(dirpath,filename)
            #print(filepath)
            # Create a variable name using the relative path
            relative_path = os.path.relpath(root, dirpath)
            #print(relative_path)
            variable_name = os.path.join(relative_path, os.path.splitext(filename)[0])
            #print(variable_name)
            #Read in the MID file
            if fnmatch.fnmatch(filename,"*MID*.tsv"):
                mid = pd.read_csv(filepath, sep="\t", skiprows=lambda x: x<=11, on_bad_lines="skip")
                mid = timeformat(mid, MID_datetime)

                # Create a variable name using the base name of the MID filename
                #variable_name = os.path.splitext(filename)[0]

                # Store the MID dataframe using the variable name
                dataframes[variable_name] = mid

            #Read in the vaclog file
            elif fnmatch.fnmatch(filename,"*vaclog*"):
                vaclog = elapsed_time(pd.read_csv(filepath, sep="\t"), "Time", vaclog_datetime)
                print(vaclog)
                
                # Create a variable name using the base name of the MID filename
                #variable_name = os.path.splitext(filename)[0]

                # Store the vaclog dataframe using the variable name
                dataframes[variable_name] = vaclog

            #Read in the hivolta file
            elif fnmatch.fnmatch(filename,"*hv log*"):
                hv = pd.read_csv(filepath, sep=",")
                hv["hv_grid"] = [abs(element) * 1e-6 for element in hv["IMon1"]]
                hv["I_em"] = [abs(element) * 1e-6 for element in hv["IMon2"]]
                hv["datetime"] = hv['Date'] + ' ' + hv['Time']
                hv = elapsed_time(hv, "datetime", hv_datetime)

                # Create a variable name using the base name of the MID filename
                #variable_name = os.path.splitext(filename)[0]
                
                # Store the hv dataframe using the variable name
                dataframes[variable_name] = hv   
            
    return dataframes




## Data processing

In [5]:
#Accessing the files
root = os.getcwd()
data = process_data(root)

# Access the processed dataframes using the variable names
for variable_name, dataframe in data.items():
    print(variable_name)
    print(dataframe)
    print("-----------------")

0        08-12-2022 11:07:05
1        08-12-2022 11:07:12
2        08-12-2022 11:07:20
3        08-12-2022 11:07:27
4        08-12-2022 11:07:34
                ...         
11420    09-12-2022 09:55:01
11421    09-12-2022 09:55:08
11422    09-12-2022 09:55:15
11423    09-12-2022 09:55:23
11424    09-12-2022 09:55:30
Name: Time, Length: 11425, dtype: object
               Live comments                Time  Elapsed time  \
0      starting lhe transfer 2022-12-08 11:07:05           0.0   
1                        NaN 2022-12-08 11:07:12           7.0   
2                        NaN 2022-12-08 11:07:20          15.0   
3                        NaN 2022-12-08 11:07:27          22.0   
4                        NaN 2022-12-08 11:07:34          29.0   
...                      ...                 ...           ...   
11420                    NaN 2022-12-09 09:55:01       82076.0   
11421                    NaN 2022-12-09 09:55:08       82083.0   
11422                    NaN 2022-12-09 09:55:

#### CernOx R-T conversion 

In [6]:
def tempconvert(df):
    new_df = df.copy()
    #Temperature curve for CERNOX - for temp stability
    A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

    #specify fit parameters A, data (Resistance values)
    def polyfit(param,data):
        total=[]
        for j in data: 
            exp=0
            for i in range(len(param)):
                exp += (param[i]/(math.log10(j))**i)
            total.append(10**exp)
        return(total)
    col_loc = int(df.columns.get_loc("T-CERNOX") + 1)    
    new_df.insert(col_loc, "CernOx Temp", polyfit(A, new_df["T-CERNOX"]))
    return new_df


#### LHe level

In [7]:
def linfit(x, a, b):
        return a * x + b
def lhe_fit(y_val, x_val): #Normal fit without extended x-axis
    # Curve fit for the data
    params, cov = curve_fit(linfit, x_val, y_val)

    # Straight line parameters
    a, b = params

    # Calculate the number of data points needed to reach b/a condition
    num_points = int(abs(b / a) / (x_val[1] - x_val[0])) + 1

    # Extend the fitted line to y_fit = 0
    x_fit = np.linspace(x_val[0], (-b/a), num_points)

    y_fit = linfit(x_fit, a, b)

    return x_fit, y_fit, a, b

def ext_fit(y_val, x_val, b_val): #extended fit with longer x-axis, user specified b-parameter
    # Curve fit for the data
    params, cov = curve_fit(linfit, x_val, y_val)

    # Straight line parameters
    a, b = params

    # Calculate the number of data points needed to reach b/a condition
    num_points = int(abs(b_val / a) / (x_val[1] - x_val[0])) + 1

    # Extend the fitted line to y_fit = 0
    x_fit_extended = np.linspace(x_val[0], (-b_val/a), num_points)

    y_fit_extended = linfit(x_fit_extended, a, b_val)

    return x_fit_extended, y_fit_extended, a, b


#### Coefficients for gauge readings
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [8]:
#Thermal transpiration
T2 = 4.2
T1 = 293
p_coef = np.sqrt(T1/T2)
print(p_coef)
#N2 to H2 equivalent conversion
CF_h2 = 2.49
def gauge_correction(df):
    new_df = df.copy()
    #apply correction factors
    ba1 = df["Barion_1"]*CF_h2/p_coef
    ba2 = df["Barion_2"]*CF_h2/p_coef
    print(ba1[0])
    print(df["Barion_1"][0])
    new_df["Barion_1"] = ba1
    new_df["Barion_2"] = ba2
    new_df.rename(columns={"Barion_1": "Barion_1 corr", "Barion_2": "Barion_2 corr"},inplace=True)
    print(new_df.head)
    return new_df

8.352359233288805


#### Calculating number of molecules injected
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

In [9]:
#constants 
V_inj = 6.515e-5
S_sample= 276

#Getting the number of molecules N and (homogenous) coverage:
def N_inj(df, inj_gauge):
    #Calculate dp
    dp = np.diff(df[inj_gauge]*100).clip(0) #mbar to Pa conversion
    # Append a 0 to the beginning of the dp array
    dp= np.insert(dp, 0, 0)
    N = (dp.cumsum())*V_inj/(scipy.constants.k*T1)
    cov = N/S_sample #Molecules per cm2
    col_loc = int(df.columns.get_loc(inj_gauge) + 1)
    df.insert(col_loc,"Number of molecules injected",N)
    df.insert(col_loc+1,"Coverage",cov)
    return df

#### Comments for annotations

In [10]:
def comments(dataframe,timecol,commentcol):
    #print hv comments
    #print(pd.unique(dataframe[colname]))
    new_dataframe = dataframe.loc[:,[timecol,commentcol]]
    #delete excess comments (hv log program writes each comment 4x)

    # Find the indices of the first occurrence of each unique event
    first_unique_indices = new_dataframe.drop_duplicates(subset=commentcol, keep="first").index

    #replace the comments in rows that are not the first occurrence of each unique event with NaN values
    new_dataframe.loc[~new_dataframe.index.isin(first_unique_indices), commentcol] = np.nan

    #drop NaN values
    new_dataframe.dropna(inplace=True)
    print(new_dataframe.index)
    return new_dataframe

#### Electron dose calculations (only for isotherm + ESD mixed measurements)

Find:

$$
Q = \int_{t_1}^{t_2} I \,dt
$$

Where $t_2$ and $t_1$ are the timestamps at which an emission current reading was taken


In [11]:
""" def electron_dose(hv):
    #define the n and n-1
    t_delta = hv["Elapsed time"].diff(periods=1).fillna(0)

    #dealing with missing datapoints and false integration times
    #Compare t_delta and I_em values: if I_em values between t_delta indexes n, n-1 are negligible, ignore this time period in integration





    return hv """

' def electron_dose(hv):\n    #define the n and n-1\n    t_delta = hv["Elapsed time"].diff(periods=1).fillna(0)\n\n    #dealing with missing datapoints and false integration times\n    #Compare t_delta and I_em values: if I_em values between t_delta indexes n, n-1 are negligible, ignore this time period in integration\n\n\n\n\n\n    return hv '

#### Simpson's rule for integration (only for isotherm + ESD mixed measurements)

$$ 
I_{Simps} = \frac{h}{3}\left(y_0+2 \sum_{i=1}^{n/2-1} y_{2i}+4 \sum_{i=1}^{n/2} y_{2i-1+y_n}\right)
$$

In [12]:
def electron_dose(df, threshold):
    # Find the indices where the emission current is above the threshold
    mask_above_threshold = df['I_em'] > threshold
    print()
    # Initialize variables
    Q = 0
    start_idx = None
    end_idx = None

    for idx, row in df.iterrows():
        print(mask_above_threshold[idx])
        if mask_above_threshold[idx]:
            if start_idx is None:
                start_idx = idx
            end_idx = idx
        else:
            if start_idx is not None:
                end_idx = idx - 1

                # Apply Simpson's rule to the subset
                subset = df.loc[start_idx:end_idx]
                subset_size = len(subset)
                h = subset['t'].diff().mean()

                # Ensure the subset has at least 4 data points for Simpson's rule
                if subset_size >= 4:
                    even_sum = subset['I_em'].iloc[2:subset_size-2:2].sum()
                    odd_sum = subset['I_em'].iloc[1:subset_size-1:2].sum()
                    integral = (subset['I_em'].iloc[0] + subset['I_em'].iloc[subset_size-1] + 4 * odd_sum + 2 * even_sum) * (h / 3)

                    Q += integral

                start_idx = None
                end_idx = None
    return Q


# Example usage
dfdata = {'t': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        'I_em': [200, 600, 700, 300, 800, 900, 400, 100, 50, 700, 800, 200, 500, 300, 900, 100, 200, 600, 700, 300]}
df = pd.DataFrame(dfdata)

threshold = 500

Q = electron_dose(df, threshold) #Threshold for ESD - 500nA



False
True
True
False
True
True
False
False
False
True
True
False
False
False
True
False
False
True
True
False


## Fitting (TODO)

#### ESD

Fitting the mathematical model PSD (ESD) dose dependence according to Malyshev:


$$\eta(D) = \eta_0 \cdot \left(\frac{D+D_1}{D_0+D_1} \right)^{-a}$$

Where parameters $D_0$ and $D_1$ are added to extend the applicability towards low doses in a way that the curve asymptotically approaches the constant initial ESD yield $eta_0$ as $D \rarr 0$. $D_0$ represents the dose imparted at the lowest measurable data point and $D_1$ is used to position the end of the initial plateau. The exponent α determines the steepness, here referred to as the conditioning rate.

In [13]:
#Define function for ESD dose dependence
def esd_fit(eta_0,D,D_0,D_1,a):
    return eta_0*((D+D_1)/(D_0+D_1))^(-a)

#Use curve_fit for dependent variable D
D_0 = 0


### Adsorption isotherms (DRK, Hobson)



Model describing the physisorption isotherms for low temperatures in the submonolayer region:

$\ln\Theta=-D\left(k_BT\ln\left(\frac{P}{P_0}\right)\right)^2=-B\left(R\,T\ln\left(\frac{P}{P_0}\right)\right)^2$, 

with $k_B$ the Boltzmann constant, $R$ the gas constant, $D$ or $B$ an empirical constant linked to the vaporisation heat of the adsorbate (DR energy)  and $P_0$ the saturated vapor pressure at the temperature $T$.

#### Hobson, 1995
Hobson took $B$ to be $B=3.59\cdot 10^{-6}\,\mathrm{cal^{-2}mol^{2}}$ following Halama and Aggus (https://doi.org/10.1116/1.568583 , 1975).
#### Chill, Wilfert and Bozyk, 2019
Chill, Wilfert and Bozyk measured isotherms between 7 and 18 K on stainless steel and found by fitting $D=3075\,\mathrm{eV}^{-2}$.

#### E. Wallen measurement data

In [14]:
#reading in other data
p_0 = 4.25e-5 #(Pa)
p_0_mbar=p_0/100
p_0_mbar
S_m = 2.39e15 #molecules/cm2
d_wallen = 3.61e4/scipy.constants.e**2 # ev^-2 to J^-2
def read_data(filepath,delim):
    df=pd.read_csv(filepath, delimiter=delim)#for wallen, use "\t"
    return df

wallen = read_data(r'C:\Users\etiirine\cernbox\Documents\etiirinen\Python\H2 Adsorption isotherms\H2 adsorption isotherm on HiLumi sample\Wallen.csv',"\t")


In [15]:
#### Fleur's data
fleur = read_data(r'C:\Users\etiirine\cernbox\Documents\etiirinen\Python\H2 Adsorption isotherms\H2 adsorption isotherm on HiLumi sample\fleurs_measurements.csv', ",")


#### Theoretical isotherms

In [16]:
#constants
b_hobson=1./528**2
gas_constant=1.9865
d_CWB = 3075/scipy.constants.e**2 # J^-2
#Hobson and DRK isotherms
def theta_hobson(p):
    return S_m*np.exp(-b_hobson*(gas_constant*4.2*np.log(p/p_0))**2)
def theta_drk(p,d_const):
    return S_m*np.exp(-d_const*(scipy.constants.k*4.2*np.log(p/p_0))**2)
fit_values = np.logspace(-18,-4,50) #Pa
thetas=np.logspace(-7,0,100)

## Calling the data analysis functions and writing to file

#### Defining variables to the called functions

In [17]:

# Defining a variable for original measurement data
vaclog = data[".\\vaclog"]
#Cut values before injection start
def rows_from_started_comment(dataframe, start_comment):
    # Find the index where the comment "started injection" is first found
    start_index = dataframe.loc[dataframe["Live comments"] == start_comment].index.min()

    # Extract all rows from the start_index to the end of the DataFrame
    extracted_rows = dataframe.loc[start_index:]

    return extracted_rows

#Calculate Number of molecules and coverage, CernOx temperature, apply gauge correction
vaclog_new = N_inj(rows_from_started_comment((gauge_correction(tempconvert(vaclog))),"started injection"),"injection 100mbar")

#LHe data
y_val = vaclog_new["helium"].values
x_val = vaclog_new["Elapsed time"].values
b_full = 560
#Straight line fit for LHe data
x_fit, y_fit, a, b = lhe_fit(y_val, x_val)
#Extended fit for LHe data
x_fit_extended, y_fit_extended, a_ext, b_full = ext_fit(y_val, x_val, b_full)

print(vaclog_new.head())

#Comments
vaclog_comments = comments(vaclog,"Time", "Live comments")
    

4.2631068666307206e-10
1.43e-09
<bound method NDFrame.head of                Live comments                Time  Elapsed time  \
0      starting lhe transfer 2022-12-08 11:07:05           0.0   
1                        NaN 2022-12-08 11:07:12           7.0   
2                        NaN 2022-12-08 11:07:20          15.0   
3                        NaN 2022-12-08 11:07:27          22.0   
4                        NaN 2022-12-08 11:07:34          29.0   
...                      ...                 ...           ...   
11420                    NaN 2022-12-09 09:55:01       82076.0   
11421                    NaN 2022-12-09 09:55:08       82083.0   
11422                    NaN 2022-12-09 09:55:15       82090.0   
11423                    NaN 2022-12-09 09:55:23       82098.0   
11424                    NaN 2022-12-09 09:55:30       82105.0   

       injection 100mbar  Barion_2 corr  Barion_1 corr  DUAL experiment  \
0                 78.515   1.422029e-09   4.263107e-10     4.996000e-0

ValueError: array must not contain infs or NaNs

In [ ]:
#### Writing to excel file
def write_to_excel(filename):
    #Insert Lhe fit data
    lhe_fit_data = {f"Fit x a={a:.4f}, b={b:.4f}":x_fit,f"Fit y a={a:.4f}, b={b:.4f}":y_fit}
    lhe_ext_fit_data = {f"Extended fit x a={a_ext:.4f}, b={b_full:.4f}":x_fit_extended,f"Extended fit y a={a_ext:.4f}, b={b_full:.4f}":y_fit_extended}
    lhe_fit_df = pd.DataFrame(lhe_fit_data)
    lhe_fit_ext_df = pd.DataFrame(lhe_ext_fit_data)
    
    #Write to excel
    writer = pd.ExcelWriter(os.path.join(os.getcwd(),filename),engine="xlsxwriter")
    vaclog.to_excel(writer,sheet_name='Original data')
    vaclog_new.to_excel(writer, sheet_name='Analysed data')
    lhe_fit_df.to_excel(writer,sheet_name="LHe fit data")
    lhe_fit_ext_df.to_excel(writer,sheet_name="LHe extended fit data")
    writer.save()
    return



## Plotting (To complete)

In [ ]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")

plt.rcParams["figure.figsize"] = [8,5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35

formatter = md.DateFormatter('%H:%M')

# Create the "graphs" folder path
graphs_folder = os.path.join(root, 'graphs')

# Ensure the "graphs" folder exists, if not create it
if not os.path.exists(graphs_folder):
    os.makedirs(graphs_folder)
    

In [ ]:
# Plotting the temp evolution
plt.figure()
plt.plot(vaclog_new["Time"],vaclog_new["CernOx Temp"],marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution")
plt.yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    

plt.savefig(os.path.join(graphs_folder,"Temp.png"),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# Helium level
fig, ax = plt.subplots()
ax.plot(x_val/3600, y_val, marker=".", markersize=5, label='inj volume pressure')
ax.plot(x_fit/3600, y_fit, "--", color="red", label=f"Fitted line (mm/s): y = {a:.4f}*x+{b:.2f}")
ax.legend(loc="upper right")
ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('LHe level (mm)')
ax.set_yscale('linear')

plt.title("Helium level drop and fit")
plt.savefig(os.path.join(graphs_folder, 'He_level.png'), dpi=300,bbox_inches='tight')
plt.show()
    

In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(data[".\\vaclog"]["Time"],data[".\\vaclog"]["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(data[".\\vaclog"]["Time"],data[".\\vaclog"]["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(data[".\\vaclog"]["Time"],data[".\\vaclog"]["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend(loc="lower right")
plt.title("Ads isotherm gauge readings as read")
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    

#Writing comments as plot annotations
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comments.index:
    plt.text(vaclog_comments["Time"][i],0.8,str(vaclog_comments["Live comments"][i]),fontdict=font, transform=trans)      

plt.savefig(os.path.join(graphs_folder, './H2 adsorption isotherm_N2 equiv pressures.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the thermal transpiration corrected H2 equiv. gauge readings
plt.figure(figsize=(12,6))
plt.plot(vaclog_new["Time"],vaclog_new["Barion_1 corr"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog_new["Time"],vaclog_new["Barion_2 corr"],marker=".", markersize=4,label='Barion 2')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend(loc="lower right")
plt.title("Ads isotherm gauge readings H2 equiv")
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right') 
       
#Writing comments as plot annotations
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comments.index:
    plt.text(vaclog_comments["Time"][i],0.8,str(vaclog_comments["Live comments"][i]),fontdict=font, transform=trans)      

plt.savefig(os.path.join(graphs_folder, 'H2 adsorption isotherm_h2 equiv Pressures.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the adsorption isotherm in low cov range
fig, ax = plt.subplots()

ax.plot(vaclog_new["Coverage"],vaclog_new["Barion_2 corr"],marker=".", markersize=5,label='h2 equiv, thermal transp corrected')
ax.legend(loc="upper left")
ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
ax.set_xlim(0, 5e16)
plt.title("H2 adsorption isotherm, zoomed-in to low cov")

plt.savefig(os.path.join(graphs_folder, 'Low coverage H2 adsorption isotherm.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the adsorption isotherms up to 1e16 M/cm2
fig, ax = plt.subplots()

#my data
ax.plot(vaclog_new["Coverage"],vaclog_new["Barion_2 corr"],marker=".", markersize=5,label='my isotherm, continuous injection')

#E. Wallen's data
ax.plot(wallen["Surface coverage [molecules per cm^2]"],wallen["Pressure [mbar]"],marker=".", markersize=5,label='E. Wallen isotherm')

#Fleur's data TODO
ax.plot(fleur["Coverage (molecules/cm^2).2"],fleur["Pressure Barion gauge (mbar)"],marker=".", markersize=5,label='Fleurs isotherm')
ax.set_xlim(0, 1e17)

ax.legend(loc="lower right")
ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
plt.title("H2 adsorption isotherm comparison")

plt.savefig(os.path.join(graphs_folder, 'H2 adsorption isotherms comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the adsorption isotherms up to 1e16 M/cm2
fig, ax = plt.subplots()

#my data
ax.plot(vaclog_new["Coverage"],vaclog_new["Barion_2 corr"],marker=".", markersize=5,label='my isotherm, continuous injection')

ax.legend(loc="upper left")
ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
plt.title("H2 adsorption isotherm")

plt.savefig(os.path.join(graphs_folder, 'My adsorption isotherm.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the theoretical isotherms +  
fig, ax = plt.subplots()

ax.plot(theta_drk(fit_values,d_CWB),fit_values,marker=".", markersize=5,label='Chill, Wilfert, Bozyk isotherm')
ax.plot(theta_hobson(fit_values),fit_values,marker=".", markersize=5,label='Hobson isotherm')
ax.plot(theta_drk(wallen["Pressure [mbar]"]*100,d_wallen),wallen["Pressure [mbar]"]*100,marker=".", markersize=5,label='Wallen isotherm data fit')

ax.legend(loc="upper left")
ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (Pa)')
ax.set_yscale('log')
plt.title("Theoretical H2 adsorption isotherm fits")

plt.savefig(os.path.join(graphs_folder, 'Theoretical isotherms.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Plotting the theoretical isotherms +  
fig, ax = plt.subplots()

ax.plot(theta_drk(fit_values,d_CWB),fit_values,marker=".", markersize=5,label='Chill, Wilfert, Bozyk isotherm')
ax.plot(theta_hobson(fit_values),fit_values,marker=".", markersize=5,label='Hobson isotherm')
ax.plot(theta_drk(wallen["Pressure [mbar]"]*100,d_wallen),wallen["Pressure [mbar]"]*100,marker=".", markersize=5,label='Wallen isotherm data fit')
ax.plot(theta_drk(vaclog_new["Barion_2 corr"]*100,d_wallen),vaclog_new["Barion_2 corr"]*100,marker=".", markersize=5,label='my data fit, using Wallen isoterm')
ax.set_xlim(left=1e15, right= 2.5e15)
ax.set_ylim(bottom=1e-17, top=1e-4)
ax.legend(loc="upper left")
ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (Pa)')
ax.set_yscale('log')
plt.title("My fit + theoretical fits")

plt.savefig(os.path.join(graphs_folder, 'my fit_Theoretical isotherms.png'), dpi=300, bbox_inches='tight')
plt.show()

## Execute writing data to file

In [ ]:
write_to_excel("adsorption isotherms_Tiirinen.xlsx")